# Baysian Networks prediction function

In [87]:
# directory
directory <- "/Users/emmafoessing/Documents/Master/MA/Code/Master-Thesis"

### Packages

In [88]:
list_of_packages <- c ("synthpop", "insight", "party", "haven", "dplyr", "rpart", "rpart.plot", "randomForest", "pROC", "caret", "pracma", "here", "Hmisc", "purrr", "randomForest", "ranger", "bnlearn", "arulesCBA", "network", "igraph")

install_if_missing <- function(p){
  if(!requireNamespace(p, quietly = TRUE)){
    install.packages(p)
  }
  library(p, character.only=TRUE)
}


lapply(list_of_packages, install_if_missing)

[[1]]
 [1] "igraph"       "network"      "arulesCBA"    "arules"       "Matrix"      
 [6] "bnlearn"      "ranger"       "purrr"        "Hmisc"        "here"        
[11] "pracma"       "caret"        "lattice"      "ggplot2"      "pROC"        
[16] "randomForest" "rpart.plot"   "rpart"        "dplyr"        "haven"       
[21] "party"        "strucchange"  "sandwich"     "zoo"          "modeltools"  
[26] "stats4"       "mvtnorm"      "grid"         "insight"      "synthpop"    
[31] "stats"        "graphics"     "grDevices"    "utils"        "datasets"    
[36] "methods"      "base"        

[[2]]
 [1] "igraph"       "network"      "arulesCBA"    "arules"       "Matrix"      
 [6] "bnlearn"      "ranger"       "purrr"        "Hmisc"        "here"        
[11] "pracma"       "caret"        "lattice"      "ggplot2"      "pROC"        
[16] "randomForest" "rpart.plot"   "rpart"        "dplyr"        "haven"       
[21] "party"        "strucchange"  "sandwich"     "zoo"          "modeltools"  
[26] "stats4"       "mvtnorm"      "grid"         "insight"      "synthpop"    
[31] "stats"        "graphics"     "grDevices"    "utils"        "datasets"    
[36] "methods"      "base"        

[[3]]
 [1] "igraph"       "network"      "arulesCBA"    "arules"       "Matrix"      
 [6] "bnlearn"      "ranger"       "purrr"        "Hmisc"        "here"        
[11] "pracma"       "caret"        "lattice"      "ggplot2"      "pROC"        
[16] "randomForest" "rpart.plot"   "rpart"        "dplyr"        "haven"       
[21] "party"        "strucchange"  "sandwich"     "zoo"          "modeltools"  
[26] "stats4"       "mvtnorm"      "grid"         "insight"      "synthpop"    
[31] "stats"        "graphics"     "grDevices"    "utils"        "datasets"    
[36] "methods"      "base"        

[[4]]
 [1] "igraph"       "network"      "arulesCBA"    "arules"       "Matrix"      
 [6] "bnlearn"      "ranger"       "purrr"        "Hmisc"        "here"        
[11] "pracma"       "caret"        "lattice"      "ggplot2"      "pROC"        
[16] "randomForest" "rpart.plot"   "rpart"        "dplyr"        "haven"       
[21] "party"        "strucchange"  "sandwich"     "zoo"          "modeltools"  
[26] "stats4"       "mvtnorm"      "grid"         "insight"      "synthpop"    
[31] "stats"        "graphics"     "grDevices"    "utils"        "datasets"    
[36] "methods"      "base"        

[[5]]
 [1] "igraph"       "network"      "arulesCBA"    "arules"       "Matrix"      
 [6] "bnlearn"      "ranger"       "purrr"        "Hmisc"        "here"        
[11] "pracma"       "caret"        "lattice"      "ggplot2"      "pROC"        
[16] "randomForest" "rpart.plot"   "rpart"        "dplyr"        "haven"       
[21] "party"        "strucchange"  "sandwich"     "zoo"          "modeltools"  
[26] "stats4"       "mvtnorm"      "grid"         "insight"      "synthpop"    
[31] "stats"        "graphics"     "grDevices"    "utils"        "datasets"    
[36] "methods"      "base"        

[[6]]
 [1] "igraph"       "network"      "arulesCBA"    "arules"       "Matrix"      
 [6] "bnlearn"      "ranger"       "purrr"        "Hmisc"        "here"        
[11] "pracma"       "caret"        "lattice"      "ggplot2"      "pROC"        
[16] "randomForest" "rpart.plot"   "rpart"        "dplyr"        "haven"       
[21] "party"        "strucchange"  "sandwich"     "zoo"          "modeltools"  
[26] "stats4"       "mvtnorm"      "grid"         "insight"      "synthpop"    
[31] "stats"        "graphics"     "grDevices"    "utils"        "datasets"    
[36] "methods"      "base"        

[[7]]
 [1] "igraph"       "network"      "arulesCBA"    "arules"       "Matrix"      
 [6] "bnlearn"      "ranger"       "purrr"        "Hmisc"        "here"        
[11] "pracma"       "caret"        "lattice"      "ggplot2"      "pROC"        
[16] "randomForest" "rpart.plot"   "rpart"        "dplyr"        "haven"       
[21] "party"        "strucchange"  "sandwich"     "zoo"       

### Data

In [89]:
load(paste0(directory, "/cpspop.RData"))
adult <- read.csv(paste0(directory, "/adult_preprocessed.csv"))
# delete NAs
adult[adult == "?"] <- NA
adult <- na.omit(adult)

adult$workclass <- as.factor(adult$workclass)
adult$education <- as.factor(adult$education)
adult$marital_status <- as.factor(adult$marital_status)
adult$relationship <- as.factor(adult$relationship)
adult$race <- as.factor(adult$race)
adult$sex <- as.factor(adult$sex)
adult$native_country <- as.factor(adult$native_country)
adult$income <- as.factor(adult$income)

adult_with_cont <- adult
cps_with_cont <- cpspop

### Helper functions

In [90]:
evaluation_metrics_factor <- function(predictions, test_set) {
    # Ensure test_set is a data frame
    test_set <- as.data.frame(test_set)
    
    # Ensure both predictions and test_set$income are factors with the same levels
    predictions <- as.factor(predictions)
    reference <- as.factor(test_set$income)
    
    # Ensure levels match between predictions and reference
    levels(predictions) <- levels(reference)
    
    # Confusion matrix for the prediction on original data
    cm <- confusionMatrix(predictions, reference, mode = "everything")

    # Saving evaluation metrics
    accuracy <- cm$overall['Accuracy']
    
    if (length(levels(reference)) == 2) {
        # Binary classification
        f1 <- cm$byClass['F1']
        sens <- cm$byClass['Sensitivity']
        spec <- cm$byClass['Specificity']
    } else {
        # Multi-class classification: calculate metrics for each class and take the mean
        f1 <- mean(cm$byClass[,'F1'], na.rm = TRUE)
        sens <- mean(cm$byClass[,'Sensitivity'], na.rm = TRUE)
        spec <- mean(cm$byClass[,'Specificity'], na.rm = TRUE)
    }

    # Create the dataframe
    metrics_df <- data.frame(
        Accuracy = accuracy, 
        F1 = f1, 
        Sensitivity = sens, 
        Specificity = spec
    )
    
    return(metrics_df)
}

Ich diskretisiere alle meine continuous cars (in Intervalle einteilen) --> das geht mit der discretize function <br>
Ich will ca. 5 Kategorien maximal pro Variable haben. Wenn eine der Ausprägungen in der Varibale mindestens 1/5 aller Ausprägungnen aus macht, dann soll diese eine eigene Kategorie werden und die restlichen als Interval kategorisiert werden. Wenn es mehrere Werte gibt, die mindestens 1/5 aller Ausprägungen ausmachen, dann sollen auch diese alle jeweils eine eigene Kategorie werden und der Rest kann in Intervalle eingeteilt werden. Am Ende soll die Variable des Datensatzes überschrieben werden mit der kategorialen Variable und der Datensatz nur aus 'factor' Variablen bestehen.
2/5 sind =0 --> 3 weitere Kategorien mit Intervallen
3/5 sind =0 --> 2 weitere Kategorien mit Intervallen

In [91]:
discretize_df = function(df, breaks = 5) {
  for (var in colnames(df)) {
    # Check if the variable is not a factor
    if (!is.factor(df[[var]])) {

      # Count the frequency of each unique value
      freq_table <- table(df[[var]])

      # Calculate the proportion of zeros, ensuring NA is handled
      zero_proportion <- ifelse(!is.na(freq_table[as.character(0)]), 
                                freq_table[as.character(0)] / sum(freq_table), 
                                0)

      # Determine the number of breaks based on zero proportion
      if (zero_proportion > 4/5) {
        new_breaks = 1
      } else if (zero_proportion > 1/4) {
        new_breaks = breaks - 2
      } else if (zero_proportion > 1/5) {
        new_breaks = breaks - 1
      } else {
        new_breaks = breaks
      }
      
      # Separate zeros and non-zeros
      zero_portion = (df[[var]] == 0)
      non_zero_values = df[[var]][!zero_portion]

      # Discretize non-zero values
      if (length(non_zero_values) > 0) {
        # Calculate breaks for non-zero values
        range_values = range(non_zero_values, na.rm = TRUE)
        breaks_values = seq(range_values[1], range_values[2], length.out = new_breaks + 1)
        
        # Ensure correct number of labels are created
        labels = sapply(1:(length(breaks_values)-1), function(i) 
                        paste("(", breaks_values[i], "-", breaks_values[i+1], "]", sep=""))

        # Use cut to apply these breaks and labels
        discretized_non_zeros = cut(non_zero_values, breaks = breaks_values, labels = labels, include.lowest = TRUE)
        # Combine zero and discretized non-zeros into the original dataframe
        df[[var]] <- factor(ifelse(zero_portion, "0", as.character(discretized_non_zeros)))
      } else {
        # If all values are zero or the number of breaks is zero or negative
        df[[var]] <- factor("0")
      }
    }
  }
  return(df)
}

#### Look at the levels created

In [92]:
print_levels <- function(data) {
  factor_vars <- sapply(data, is.factor)
  for (var in names(data)[factor_vars]) {
    cat("Levels of", var, ":\n")
    print(levels(data[[var]]))
    cat("\n")
  }
}

In [93]:
# cps
discretized_data <- discretize_df(cpspop)
print_levels(discretized_data)

# adult
discretized_data <- discretize_df(adult)
print_levels(discretized_data)

Levels of tax :
[1] "(1-33333]"     "(33333-66665]" "(66665-99997]" "0"            

Levels of income :
[1] "(1-153749.2]"        "(153749.2-307497.4]" "(307497.4-461245.6]"
[4] "(461245.6-614993.8]" "(614993.8-768742]"  

Levels of csp :
[1] "(1-23917]" "0"        

Levels of age :
[1] "(15-30]" "(30-45]" "(45-60]" "(60-75]" "(75-90]"

Levels of educ :
 [1] "31" "32" "33" "34" "35" "36" "37" "38" "39" "40" "41" "42" "43" "44" "45"
[16] "46"

Levels of marital :
[1] "1" "2" "3" "4" "5" "6" "7"

Levels of race :
[1] "1" "2" "3" "4"

Levels of sex :
[1] "1" "2"

Levels of ss :
[1] "(16671.3333333333-33335.6666666667]" "(33335.6666666667-50000]"           
[3] "(7-16671.3333333333]"                "0"                                  

Levels of age :
[1] "(17-31.6]"   "(31.6-46.2]" "(46.2-60.8]" "(60.8-75.4]" "(75.4-90]"  

Levels of workclass :
[1] "0" "1" "2" "3" "4" "5" "6"

Levels of fnlwgt :
[1] "(1190517.8-1484705]"  "(13769-307956.2]"     "(307956.2-602143.4]" 
[4] "(602143.4-8963

In [94]:
cpdag_to_dag <- function(cpdag) {
  # Convert bnlearn object to adjacency matrix
  adj_matrix <- amat(cpdag)
  
  # Convert adjacency matrix to igraph object
  ig <- graph_from_adjacency_matrix(adj_matrix, mode = "directed")
  
  # Check if it's already a DAG
  if (igraph::is_dag(ig)) {
    return(cpdag)
  }
  
  # Convert CPDAG to DAG by randomly orienting undirected edges
  directed_arcs <- directed.arcs(cpdag)
  undirected_arcs <- undirected.arcs(cpdag)
  
  while (nrow(undirected_arcs) > 0) {
    arc <- undirected_arcs[1, , drop = FALSE]
    cpdag <- set.arc(cpdag, from = arc[1, 1], to = arc[1, 2])
    undirected_arcs <- undirected.arcs(cpdag)
  }
  return(cpdag)
}

In [95]:
train_bn <- function(data, algorithm, score = NULL) {
  if (algorithm == "hc") {
    bn <- bnlearn::hc(data, score = score)
  } else if (algorithm == "tabu") {
    bn <- bnlearn::tabu(data, score = score)
  } else if (algorithm == "gs") {
    bn <- bnlearn::gs(data)
    bn <- bnlearn::cpdag(bn) # Convert to a completed partially directed acyclic graph
    bn <- cpdag_to_dag(bn) # Convert CPDAG to DAG
  } else if (algorithm == "iamb") {
    bn <- bnlearn::iamb(data)
    bn <- bnlearn::cpdag(bn) # Convert to a completed partially directed acyclic graph
    bn <- cpdag_to_dag(bn) # Convert CPDAG to DAG
  } else {
    stop("Unsupported algorithm")
  }

  cat(algorithm, "\n")

  bn.fit(bn, data)
}

In [96]:
# Define a function to evaluate the model using cross-validation
evaluate_bn <- function(data, bn_fitted, target_var) {
  predictions <- predict(bn_fitted, data = data, node = target_var)
  actual <- data[[target_var]]
  accuracy <- mean(predictions == actual)
  return(accuracy)
}

# Just BN prediction on original data

In [133]:
# Define discretize_df function
discretize_df = function(df, breaks = 5) {
  for (var in colnames(df)) {
    # Check if the variable is not a factor
    if (!is.factor(df[[var]])) {

      # Count the frequency of each unique value
      freq_table <- table(df[[var]])

      # Calculate the proportion of zeros, ensuring NA is handled
      zero_proportion <- ifelse(!is.na(freq_table[as.character(0)]), 
                                freq_table[as.character(0)] / sum(freq_table), 
                                0)

      # Determine the number of breaks based on zero proportion
      if (zero_proportion > 4/5) {
        new_breaks = 1
      } else if (zero_proportion > 1/4) {
        new_breaks = breaks - 2
      } else if (zero_proportion > 1/5) {
        new_breaks = breaks - 1
      } else {
        new_breaks = breaks
      }
      
      # Separate zeros and non-zeros
      zero_portion = (df[[var]] == 0)
      non_zero_values = df[[var]][!zero_portion]

      # Discretize non-zero values
      if (length(non_zero_values) > 0) {
        # Calculate breaks for non-zero values
        range_values = range(non_zero_values, na.rm = TRUE)
        breaks_values = seq(range_values[1], range_values[2], length.out = new_breaks + 1)
        
        # Ensure correct number of labels are created
        labels = sapply(1:(length(breaks_values)-1), function(i) 
                        paste("(", breaks_values[i], "-", breaks_values[i+1], "]", sep=""))

        # Use cut to apply these breaks and labels
        discretized_non_zeros = cut(non_zero_values, breaks = breaks_values, labels = labels, include.lowest = TRUE)
        # Combine zero and discretized non-zeros into the original dataframe
        df[[var]] <- factor(ifelse(zero_portion, "0", as.character(discretized_non_zeros)))
      } else {
        # If all values are zero or the number of breaks is zero or negative
        df[[var]] <- factor("0")
      }
    }
  }
  return(df)
}

# Define cpdag_to_dag function
cpdag_to_dag <- function(cpdag) {
  adj_matrix <- amat(cpdag)
  ig <- graph_from_adjacency_matrix(adj_matrix, mode = "directed")
  if (igraph::is_dag(ig)) {
    return(cpdag)
  }
  directed_arcs <- directed.arcs(cpdag)
  undirected_arcs <- undirected.arcs(cpdag)
  while (nrow(undirected_arcs) > 0) {
    arc <- undirected_arcs[1, , drop = FALSE]
    cpdag <- set.arc(cpdag, from = arc[1, 1], to = arc[1, 2])
    undirected_arcs <- undirected.arcs(cpdag)
  }
  return(cpdag)
}

train_bn <- function(data, algorithm) {
  if (any(is.na(data))) {
    stop("The data contains missing values.")
  }
  
  # Train the Bayesian network using the specified algorithm with default BIC score
  if (algorithm =="hc") {
    bn <- bnlearn::hc(data)  # Using BIC by default
  } else if (algorithm == "tabu") {
    bn <- bnlearn::tabu(data)  # Using BIC by default
  } else if (algorithm == "gs") {
    bn <- bnlearn::gs(data)
    bn <- bnlearn::cpdag(bn)  # Convert to CPDAG
    bn <- cpdag_to_dag(bn)     # Convert CPDAG to DAG
  } else if (algorithm == "iamb") {
    bn <- bnlearn::iamb(data)
    bn <- bnlearn::cpdag(bn)  # Convert to CPDAG
    bn <- cpdag_to_dag(bn)     # Convert CPDAG to DAG
  } else {
    stop("Unsupported algorithm.")
  }
  
  return(bn)
}


# Define a function to evaluate the Bayesian network model
evaluate_bn <- function(testData, bn_fitted, target_var) {
  # Generate predictions for the target variable
  predictions <- predict(bn_fitted, data = testData, node = target_var)
  
  # Compare the predictions with the actual values in the test data
  actual_values <- testData[[target_var]]
  
  # Calculate the accuracy
  accuracy <- mean(predictions == actual_values, na.rm = TRUE)
  
  # You can also calculate other metrics such as:
  confusion <- table(Predicted = predictions, Actual = actual_values)
  precision <- diag(confusion) / rowSums(confusion)
  recall <- diag(confusion) / colSums(confusion)
  f1_score <- 2 * (precision * recall) / (precision + recall)
  
  # Return a list of evaluation metrics
  return(list(
    accuracy = accuracy,
    precision = precision,
    recall = recall,
    f1_score = f1_score
  ))
}


In [137]:
# Define the custom model list
bn_model <- list(
  label = "Bayesian Network",
  library = "bnlearn",
  type = "Classification",
  
  parameters = data.frame(
    parameter = c("algorithm"),
    class = c("character"),
    label = c("Algorithm")
  ),
  
  grid = function(x, y, len = NULL, search = "grid") {
    algorithms <- c("hc", "tabu", "gs", "iamb")
    
    if (search == "grid") {
      expand.grid(algorithm = algorithms)
    } else {
      data.frame(algorithm = sample(algorithms, len, replace = TRUE))
    }
  },
  
  fit = function(x, y, wts, param, lev, last, classProbs, ...) {
  df <- as.data.frame(x)
  df$income <- y
  
  # Ensure consistent factor levels across folds
  df <- lapply(df, function(col) {
    if (is.factor(col)) {
      levels(col) <- union(levels(col), unique(col))
    }
    return(col)
  })
  df <- as.data.frame(df)
  
  tryCatch({
    # Train the Bayesian network using the specified algorithm
    bn <- train_bn(df, param$algorithm)
    
    # Fit the parameters of the Bayesian network
    bn_fitted <- bn.fit(bn, df)
    return(bn_fitted)
  }, error = function(e) {
    print(paste("Error in fitting model:", e$message))
    return(NULL)
  })
},
  
  predict = function(modelFit, newdata, preProc = NULL, submodels = NULL) {
    if (is.null(modelFit)) {
      return(rep(NA, nrow(newdata)))
    }
    
    data <- discretize_df(newdata)
    predictions <- tryCatch({
      predict(modelFit, data)
    }, error = function(e) {
      print(paste("Error in prediction: ", e$message))
      return(rep(NA, nrow(newdata)))
    })
    
    return(predictions)
  },
  
  prob = NULL,  # Bayesian Networks in bnlearn do not directly provide class probabilities
  
  predictors = function(x, ...) {
    colnames(x)
  },
  
  sort = function(x) x,
  
  levels = function(x) levels(x$obs)
)



In [140]:
bn_pred <- function(data, outer_folds, inner_folds) {
  # Discretize the data
  data <- discretize_df(data)
  
  # Ensure all variables are factors
  data <- data.frame(lapply(data, function(x) {
    if (!is.factor(x)) {
      x <- factor(x)
    } else {
      x <- factor(x, levels = unique(x))
    }
    return(x)
  }))
  
  outer_control <- trainControl(
    method = "cv", number = outer_folds,
    summaryFunction = multiClassSummary,
    verboseIter = FALSE,
    allowParallel = TRUE
  )
  
  inner_control <- trainControl(
    method = "cv", number = inner_folds,
    summaryFunction = multiClassSummary,
    verboseIter = FALSE,
    allowParallel = TRUE
  )
  
  tunegrid <- expand.grid(algorithm = c("hc", "tabu", "gs", "iamb"))#,
                          #score = c("aic", "bic"))
  
  outer_results <- list()
  outer_cv_folds <- createFolds(data$income, k = outer_folds)
  
  for (i in seq_along(outer_cv_folds)) {
    outer_test_index <- outer_cv_folds[[i]]
    outer_testData <- data[outer_test_index,]
    outer_trainData <- data[-outer_test_index,]
    
    # Check the structure of the training and testing data
    print(paste("NA values in outer_trainData for fold", i, ":", any(is.na(outer_trainData))))
    print(paste("NA values in outer_testData for fold", i, ":", any(is.na(outer_testData))))
    print(paste("Structure of outer_trainData for fold", i, ":"))
    print(str(outer_trainData))
    
    model <- train(income ~ ., 
                   data = outer_trainData, 
                   method = bn_model,
                   tuneGrid = tunegrid, 
                   trControl = inner_control)
    
    if (is.null(model$bestTune)) {
      print(paste("Skipping fold", i, "due to failed model fitting"))
      next
    }
    
    best_hyperparameters <- model$bestTune
    
    final_model <- train(income ~ ., 
                         data = outer_trainData, 
                         method = bn_model,
                         trControl = outer_control, 
                         tuneGrid = best_hyperparameters)
    
    predictions <- predict(final_model, newdata = outer_testData)
    eval_metrics <- evaluate_bn(testData = outer_testData, bn_fitted = final_model$finalModel, target_var = "income")
    outer_results[[i]] <- eval_metrics
  }
  
  eval_avg_outer_folds <- do.call(rbind, outer_results) %>%
    summarise(across(everything(), mean, na.rm = TRUE))
  
  return(eval_avg_outer_folds)
}

In [141]:
cps_res <- bn_pred(cps_with_cont, 2, 2)

[1] "NA values in outer_trainData for fold 1 : FALSE"
[1] "NA values in outer_testData for fold 1 : FALSE"
[1] "Structure of outer_trainData for fold 1 :"
'data.frame':	24719 obs. of  9 variables:
 $ tax    : Factor w/ 4 levels "0","(1-33333]",..: 1 2 1 1 2 1 2 1 1 2 ...
 $ income : Factor w/ 5 levels "(1-153749.2]",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ csp    : Factor w/ 2 levels "0","(1-23917]": 1 1 1 1 1 1 1 1 1 1 ...
 $ age    : Factor w/ 5 levels "(30-45]","(60-75]",..: 1 3 1 1 1 1 4 1 4 1 ...
 $ educ   : Factor w/ 16 levels "34","36","43",..: 1 3 4 6 9 10 11 11 6 12 ...
 $ marital: Factor w/ 7 levels "5","7","6","2",..: 1 1 3 2 5 5 5 5 5 2 ...
 $ race   : Factor w/ 4 levels "1","2","3","4": 1 1 1 1 1 1 2 1 2 2 ...
 $ sex    : Factor w/ 2 levels "2","1": 1 1 1 1 2 1 2 2 2 1 ...
 $ ss     : Factor w/ 4 levels "0","(7-16671.3333333333]",..: 1 1 1 1 1 1 1 1 1 1 ...
NULL


Warning message:
"model fit failed for Fold1: algorithm=hc Error in check.nodes(name, x) : invalid node(s) 'xNames'.
"


In [143]:
adult_res <- bn_pred(adult, 2, 2)

[1] "NA values in outer_trainData for fold 1 : FALSE"
[1] "NA values in outer_testData for fold 1 : FALSE"
[1] "Structure of outer_trainData for fold 1 :"
'data.frame':	15081 obs. of  14 variables:
 $ age           : Factor w/ 5 levels "(75.4-90]","(46.2-60.8]",..: 2 2 3 3 3 2 4 2 3 2 ...
 $ workclass     : Factor w/ 7 levels "2","5","0","4",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ fnlwgt        : Factor w/ 5 levels "(13769-307956.2]",..: 1 1 1 1 1 2 1 1 1 1 ...
 $ education     : Factor w/ 16 levels "11","5","15",..: 2 7 8 6 9 8 1 3 11 12 ...
 $ marital_status: Factor w/ 7 levels "6","0","5","4",..: 2 1 3 2 2 2 2 5 5 5 ...
 $ occupation    : Factor w/ 6 levels "(1-3.4]","(5.8-8.2]",..: 2 1 3 5 1 5 5 1 2 5 ...
 $ relationship  : Factor w/ 6 levels "1","4","3","2",..: 2 1 1 1 1 1 2 5 1 5 ...
 $ race          : Factor w/ 5 levels "4","2","1","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ sex           : Factor w/ 2 levels "0","1": 1 1 2 2 2 2 1 2 2 2 ...
 $ capital_gain  : Factor w/ 2 levels "0","(114-99999]"

In [151]:
cps_res

,Accuracy,F1,Sensitivity,Specificity
,<dbl>,<dbl>,<dbl>,<dbl>
Accuracy,0.9606435,0.9799267,0.25,0.8


### Save the results

In [1]:
# Bind results
bn_pred_results <- list(cps_res = cps_res, adult_res = adult_res)
# File pth for output
file <- "/user/emma.foessing01/u11969/results/bn_pred_results.RData" 
dir.create(dirname(output_file), recursive = TRUE, showWarnings = FALSE) # create dir if not there
# Save the results to an RData file 
save(results, file = output_file)

ERROR: Error in eval(expr, envir, enclos): Objekt 'cps_res' nicht gefunden


In [132]:
## old train function with gridsearch over score param:

# Define train_bn function
train_bn <- function(data, algorithm, score = NULL) {
  if (any(is.na(data))) {
    stop("The data contains missing values.")
  }
  
  if (algorithm %in% c("hc", "tabu") && !is.null(score)) {
    if (algorithm == "hc") {
      bn <- bnlearn::hc(data, score = score)
    } else if (algorithm == "tabu") {
      bn <- bnlearn::tabu(data, score = score)
    }
  } else if (algorithm == "gs") {
    bn <- bnlearn::gs(data)
    bn <- bnlearn::cpdag(bn) # Convert to a completed partially directed acyclic graph
    bn <- cpdag_to_dag(bn) # Convert CPDAG to DAG
  } else if (algorithm == "iamb") {
    bn <- bnlearn::iamb(data)
    bn <- bnlearn::cpdag(bn) # Convert to a completed partially directed acyclic graph
    bn <- cpdag_to_dag(bn) # Convert CPDAG to DAG
  } else {
    stop("Unsupported algorithm or missing score for algorithm.")
  }
  
  return(bn)
}

# and old eval function
evaluate_bn <- function(testData, bn_fitted, target_var) {
  predictions <- predict(bn_fitted, data = testData, node = target_var)
  mean(predictions == testData[[target_var]])
}